In [29]:
# from utils_fs_v2 import DataGenerator_res2
from jax import grad, jit, nn
import jax.numpy as jnp
import numpy as np
# import jax.random as random
# from jax import vmap
# from jax.tree_util import tree_flatten, tree_unflatten, register_pytree_node, \
#     tree_flatten_with_path, tree_structure, tree_map
import jax.tree_util as tu
import matplotlib.pyplot as plt


# Trying to Train Neural Networks with Overlap

In [38]:
def init_mlp_params(layer_widths):
  params = []
  for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
    params.append(
        dict(weights=np.random.normal(size=(n_in, n_out)) * np.sqrt(2/n_in),
             biases=np.ones(shape=(n_out,))
            )
    )
  return params

In [39]:
# tree_map(lambda x: x.shape, params)

In [40]:
def forward(params, x):
  *hidden, last = params
  for layer in hidden:
    x = nn.relu(x @ layer['weights'] + layer['biases'])
  return x @ last['weights'] + last['biases']

def loss_fn(nn, x, y):
  return jnp.mean((forward(nn.params, x) - y) ** 2)

LEARNING_RATE = 0.0001

@jit
def update(nn, x, y):
  grad_loss_fn = grad(loss_fn)
  grads = tu.tree_map(grad_loss_fn,nn, x, y)
  # Note that `grads` is a pytree with the same structure as `params`.
  # `jax.grad` is one of the many JAX functions that has
  # built-in support for pytrees.

  # This is handy, because we can apply the SGD update using tree utils:
  return tu.tree_map(
      lambda p, g: p - LEARNING_RATE * g, nn, grads
  )

In [41]:
class ParameterContainer:
    def __init__(self):
        self.params = init_mlp_params([1, 128, 128, 1])

In [42]:
# params1 = init_mlp_params([1, 128, 128, 1])
# params2 = init_mlp_params([1, 128, 128, 1])
nn1 = ParameterContainer()
nn2 = ParameterContainer()
nn = [nn1, nn2]
xs1 = 0.5*np.random.uniform(size=(70, 1))
ys1 = xs1 ** 2
xs2 = 0.5*np.random.uniform(size=(60, 1)) + 0.5
ys2 = xs2 ** 2
xs = [xs1,xs2]
ys = [ys1,ys2]

# print(tu.tree_structure(nn))
# print(tu.tree_structure(xs))
# print(tu.tree_structure(ys))

# tu.treedef_is_leaf(params)
# type(params)

In [43]:
for _ in range(10000):
  params1 = update(nn, xs, ys)

# plt.scatter(xs, ys)
# plt.scatter(xs, forward(params1, xs), label='Model prediction')
plt.legend()

TypeError: Cannot interpret value of type <class '__main__.ParameterContainer'> as an abstract array; it does not have a dtype attribute

In [1]:
temp1 = [-1,3]
temp2 = [2,3,4]
temp3 = [0,6]
temp = [temp1,temp2,temp3]

3

In [11]:
# domain_bounds = jnp.array([0.,1.])
# single_domains = jnp.array([[0.,0.4],[0.6,1.]])
# double_domains = jnp.array([[0.4,0.6]])

domain_bounds = jnp.array([0.,2.])
single_domains = jnp.array([[0.,0.4],[0.6,1.],[1.333,2.0]])
double_domains = jnp.array([[0.4,0.6],[1.,1.333]])
D = DataGenerator_res2(domain_bounds,single_domains,double_domains,10)

In [12]:
# D.single_domain_point_dict

In [13]:
def show_example(structured):
  flat, tree = tree_flatten(structured)
  unflattened = tree_unflatten(tree, flat)
  print(f"{structured=}\n  {flat=}\n  {tree=}\n  {unflattened=}")

In [14]:
show_example(D.single_domain_point_dict)

structured=[{'res_pts': Array([[0.34768867]], dtype=float32), 'uni_pts': Array([[0.06492133]], dtype=float32)}, {'res_pts': Array([[0.84995735]], dtype=float32), 'uni_pts': Array([[0.63279486]], dtype=float32)}, {'res_pts': Array([[1.8789763]], dtype=float32), 'uni_pts': Array([[1.9840243],
       [1.5907514]], dtype=float32)}]
  flat=[Array([[0.34768867]], dtype=float32), Array([[0.06492133]], dtype=float32), Array([[0.84995735]], dtype=float32), Array([[0.63279486]], dtype=float32), Array([[1.8789763]], dtype=float32), Array([[1.9840243],
       [1.5907514]], dtype=float32)]
  tree=PyTreeDef([{'res_pts': *, 'uni_pts': *}, {'res_pts': *, 'uni_pts': *}, {'res_pts': *, 'uni_pts': *}])
  unflattened=[{'res_pts': Array([[0.34768867]], dtype=float32), 'uni_pts': Array([[0.06492133]], dtype=float32)}, {'res_pts': Array([[0.84995735]], dtype=float32), 'uni_pts': Array([[0.63279486]], dtype=float32)}, {'res_pts': Array([[1.8789763]], dtype=float32), 'uni_pts': Array([[1.9840243],
       [1.59